Basic pipeline to try causal inference from Huggingface pipeline directly

# Loading

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade
!pip install accelerate
!pip install bitsandbytes

In [4]:
#Libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:
#Use the open source available Text-Generation models in HF Model Cards
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             cache_dir='./cache',
                                             device_map='cuda:0',
                                             load_in_4bit=True
                                             #dtype = torch.float16
                                             )

## Causal Inference

In [8]:
text = '''
You are a Mathematician and for the given mathematical question return the answer as per the instructions given to you

##
Instructions :-
1. Answer the given question only and do not make any assumptions on your own.
2. Make the answer as short as possible
3. Be as precise and concise as you can be.
##

Question :-

1. Equation = {3x + 400 + 26y = 1000 + 40x}
   For the above equation give me the value for x.

2. A builder is constructing a roof.
   The wood he is using for the sloped section of the roof is 4m long and the peak of the roof needs to be 2m high.
   What angle should be the piece of wood make with the base of the roof ?

Answer :-

'''

In [9]:
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")

In [10]:
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')



You are a Mathematician and for the given mathematical question return the answer as per the instructions given to you

##
Instructions :- 
1. Answer the given question only and do not make any assumptions on your own.
2. Make the answer as short as possible
3. Be as precise and concise as you can be.
##

Question :-

1. Equation = {3x + 400 + 26y = 1000 + 40x}
   For the above equation give me the value for x.

2. A builder is constructing a roof.
   The wood he is using for the sloped section of the roof is 4m long and the peak of the roof needs to be 2m high.
   What angle should be the piece of wood make with the base of the roof ?

Answer :-

1. The value for x is 4.

2. The angle is 45


## Comprehension

In [40]:
text = '''
You are an expert question answering agent. Follow the instructions.

##
Instructions :-
1. Read the given passage under "Passage" and answer the questions asked in "Questions".
2. Be as precise and concise as you can.
3. Answer to the question should be from the given passage only and nothing else.
##

##
Passage

Mike and Morris lived in the same village.
While Morris owned the largest jewelry shop in the village, Mike was a poor farmer.
Both had large families with many sons, daughters-in-law and grandchildren.
One fine day, Mike, tired of not being able to feed his family, decided to leave the village and move to the city where he was certain to earn enough to feed everyone.
Along with his family, he left the village for the city. At night, they stopped under a large tree.
There was a stream running nearby where they could freshen up themselves.
##


Questions :-

1. What was the job of Mike ?
2. Describe family of Mike.
Answer :-

'''

In [41]:
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")

In [42]:
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


You are an expert question answering agent. Follow the instructions.

##
Instructions :- 
1. Read the given passage under "Passage" and answer the questions asked in "Questions".
2. Be as precise and concise as you can.
3. Answer to the question should be from the given passage only and nothing else.
##

##
Passage

Mike and Morris lived in the same village. 
While Morris owned the largest jewelry shop in the village, Mike was a poor farmer. 
Both had large families with many sons, daughters-in-law and grandchildren. 
One fine day, Mike, tired of not being able to feed his family, decided to leave the village and move to the city where he was certain to earn enough to feed everyone. 
Along with his family, he left the village for the city. At night, they stopped under a large tree. 
There was a stream running nearby where they could freshen up themselves. 
##


Questions :-

1. What was the job of Mike ?
2. Describe family of Mike.
Answer :-

1. Job of Mike - He was a poor farmer.
2. 

What I can see from the question answering and causal answering part is that even for such basic Q/A a small model of 1B parameters is not that good unless the LLM formatted properly.

Ultimately its a decoder only which can complete sentences, without prompting it to "Answer" it cannot give a complete answer.

Generation can make sense but not for causal inference directly.